# Optibook Manual

This notebook contains examples for **all interactions** you can do with optibook (that means inserting orders, getting your positions, etc.). You can use this notebook as a reference for the system later on when you write more extensive algorithms.

First we do some setup and import the optibook client, which is used to connect to the optibook exchange.

In [1]:
from optibook.synchronous_client import Exchange

import time
import logging
logger = logging.getLogger('client')
logger.setLevel('ERROR')

print("Setup was successful.")

Setup was successful.


## Define Your Instrument

An instrument is the term we use for the product that we are trading. As such, if we are trading BMW stocks, we would call that stock an instrument. An option on BMW would be a different instrument, and a future yet another. 

Here we set a variable to an instrument name, which we use in all functions down the line. You can see all available instruments by looking at the dropdown menu labeled "Instruments" on the visualizer.

In [2]:
instrument_id = 'TEST'

## Connect to Exchange

In [4]:
e = Exchange()
a = e.connect()

2021-11-24 09:27:40,124 [asyncio   ] [MainThread  ] Using selector: EpollSelector
2021-11-24 09:27:40,299 [client    ] [Thread-4    ] Forcing a disconnect due to an error: Closing connection because someone else logged in with the same credentials. Only one session may be active at the same time.


## Order Books

In [6]:
# Returns the current limit order book for the instrument
book = e.get_last_price_book(instrument_id)
  
if not book.bids:
    print('No bids at all for instrument.')
else:
    best_bid = book.bids[0]
    price = best_bid.price
    volume = best_bid.volume
    print(f'Best bid is {volume} lots @ price {price:.2f}.')

if not book.asks:
    print('No asks at all for instrument.')
else:
    best_ask = book.asks[0]
    price = best_ask.price
    volume = best_ask.volume
    print(f'Best ask is {volume} lots @ price {price:.2f}.')

Best bid is 20 lots @ price 400.00.
No asks at all for instrument.


## Public Trade Ticks

In [19]:
# Returns all public tradeticks since the instantiation of the Exchange (upto a max limit)
tradeticks = e.get_trade_tick_history(instrument_id)

In [20]:
# Returns all public tradeticks since the last time this function was called
tradeticks = e.poll_new_trade_ticks(instrument_id)

In [21]:
for tradetick in tradeticks:
    timestamp = tradetick.timestamp
    price = tradetick.price
    volume = tradetick.volume
    
    print(f'Tradetick at {timestamp}: {volume:4.0f} lots @ {price:.2f}')

Tradetick at 2021-11-24 09:35:55:    2 lots @ 120.00
Tradetick at 2021-11-24 09:35:55:    2 lots @ 120.00
Tradetick at 2021-11-24 09:35:55:    2 lots @ 125.00
Tradetick at 2021-11-24 09:35:55:    5 lots @ 125.00
Tradetick at 2021-11-24 09:35:55:    7 lots @ 130.00
Tradetick at 2021-11-24 09:35:55:    3 lots @ 150.00


## Private Trades

In [22]:
# Returns all trades you have done since since the instantiation of the Exchange
trades = e.get_trade_history(instrument_id)

In [23]:
# Returns all trades you have done since the last time this function was called
trades = e.poll_new_trades(instrument_id)

In [24]:
for trade in trades:
    timestamp = trade
    price = trade.price
    volume = trade.volume
    
    buysell = 'Bought' if trade.side == 'bid' else 'Sold'
    
    print(f'{buysell:6s} {volume:3.0f} lots @ {price:.2f}')

Sold     5 lots @ 120.00


## Positions & PnL

In [25]:
# Returns all current positions
positions = e.get_positions()
for iid in positions:
    print(f'{iid:18s}: {positions[iid]:3.0f}')

DEMO              :   0
TEST              :  -5
MKT1              :   0
MKT2              :   0
MKT3              :   0


In [26]:
# Returns postions with cash component
positions = e.get_positions_and_cash()
for iid in positions:
    print(f'''{iid:18s}: {positions[iid]['volume']:3.0f} position, {positions[iid]['cash']:10.2f} cash''')

DEMO              :   0 position,       0.00 cash
TEST              :  -5 position,     600.00 cash
MKT1              :   0 position,       0.00 cash
MKT2              :   0 position,       0.00 cash
MKT3              :   0 position,       0.00 cash


In [27]:
# Returns current PnL based on last traded price
pnl = e.get_pnl()
print(pnl)

-150.0


## Inserting Orders

In [13]:
# Insert bid limit order - This trades against any current orders, and any remainders become new resting orders in the book
result = e.insert_order(instrument_id, price=100, volume=5, side='bid', order_type='limit')
print(f"Order Id: {result}")

Order Id: 3986


In [14]:
# Insert ask limit order - This trades against any current orders, and any remainders become new resting orders in the book
result = e.insert_order(instrument_id, price=120, volume=5, side='ask', order_type='limit')
print(f"Order Id: {result}")

Order Id: 3987


In [29]:
# Insert bid IOC - This can trade against any resting volume but does not remain in the book
result = e.insert_order(instrument_id, price=100.0, volume=1, side='bid', order_type='ioc')
print(f"Order Id: {result}")

Order Id: 4697


In [ ]:
# Insert ask IOC - This can trade against any resting volume but does not remain in the book
result = e.insert_order(instrument_id, price=430.0, volume=1, side='ask', order_type='ioc')
print(f"Order Id: {result}")

## Order Management

In [12]:
# Returns list of outstanding (unfilled) orders
orders = e.get_outstanding_orders(instrument_id)
for order_id in orders:
    print(f'Order {order_id}: {orders[order_id].side} {orders[order_id].volume:3.0f} @ {orders[order_id].price:.2f}')

In [8]:
# Attempt to delete inserted order by order_id
order_id = 3496
result = e.delete_order(instrument_id, order_id=order_id)
print(f"Deleted order id {order_id}: {result}")

Deleted order id 3496: True


In [ ]:
# Change volume for existing order
order_id = 4
new_volume = 16
result = e.amend_order(instrument_id, order_id=order_id, volume=new_volume)
print(f"Changed volume for order id {order_id} to {new_volume} lots: {result}.")

In [11]:
# Delete all outstanding orders
outstanding = e.get_outstanding_orders(instrument_id)
for o in outstanding.values():
    result = e.delete_order(instrument_id, order_id=o.order_id)
    print(f"Deleted order id {o.order_id}: {result}")

Deleted order id 3813: True


## 'Hack' Out of Positions

In [28]:
# Get out of all positions you are currently holding, regardless of the loss involved. That means selling whatever
# you are long, and buying-back whatever you are short. Be sure you know what you are doing when you use this logic.

positions = e.get_positions()
print('Positions before:')
for iid in positions:
    print(f'{iid:18s}: {positions[iid]:3.0f}')

print(f'\nPnL before: {e.get_pnl():.2f}')
print(f'\n')

for s, p in e.get_positions().items():
    if p > 0:
        print(f'Selling {p} lots of {s}.')
        e.insert_order(s, price=1, volume=p, side='ask', order_type='ioc')
    elif p < 0:
        print(f'Buying {abs(p)} lots of {s}.')
        e.insert_order(s, price=100000, volume=-p, side='bid', order_type='ioc')
    time.sleep(0.10)
        
positions = e.get_positions()
print('\nPositions after:')
for iid in positions:
    print(f'{iid:18s}: {positions[iid]:3.0f}')

print(f'\nPnL after: {e.get_pnl():.2f}')

Positions before:
DEMO              :   0
TEST              :   0
MKT1              :   0
MKT2              :   0
MKT3              :   0

PnL before: 100.00



Positions after:
DEMO              :   0
TEST              :   0
MKT1              :   0
MKT2              :   0
MKT3              :   0

PnL after: 100.00
